In [ ]:
from jax import random
from dynonet import MimoLTI

In [ ]:
I = 1 # number of inputs
O = 1 # number of outputs
T = 1000 # number of time steps
B = 32 # batch size
na = 10
nb = 10
n_hidden = 20

In [ ]:
key = random.key(200)
key, subkey = random.split(key)

In [ ]:
import flax.linen as nn
from typing import Sequence

class MLP(nn.Module):
    features: Sequence[int]

    def setup(self):
        self.layers = [nn.Dense(feat) for feat in self.features]

    def __call__(self, inputs):
        x = inputs
        for i, lyr in enumerate(self.layers):
            x = lyr(x)
            if i != len(self.layers) - 1:
                x = nn.tanh(x)
        return x
    
class DynoNet(nn.Module):
  in_channels: int = 1
  out_channels: int = 1
  nb: int = 10
  na: int = 10
  n_hidden: int = 20 

  @nn.compact
  def __call__(self, u):
    y = nn.Sequential([MimoLTI(I, 4, nb, na),
                          MLP([n_hidden, 3]),
                          MimoLTI(3, O, nb, na)]
    )(u)
    return y + MimoLTI(I, O, nb, na)(u)



In [ ]:
key, subkey = random.split(key)
u = random.normal(subkey, (B, T, I))
model = DynoNet()
key, subkey = random.split(key)
params = model.init(subkey, u)
y1 = model.apply(params, u)
y1.shape